# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Прочитаем файлы и посмотрим первые 5 строк, так же проверим на дубликаты и пропуски

In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from scipy import stats as st

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
print(df_0.head(5))
print()
print(df_1.head(5))
print()
print(df_2.head(5))

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [4]:
print(df_0.isna().sum())
print()
print(df_1.isna().sum())
print()
print(df_2.isna().sum())

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


In [5]:
print(df_0.duplicated().sum())
print()
print(df_1.duplicated().sum())
print()
print(df_2.duplicated().sum())

0

0

0


In [6]:
row = [df_0, df_1, df_2]
for i in row:
    i.drop(i.query('id.duplicated() == True').index, inplace=True)

In [7]:
df_0

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


### Вывод: 
Мы открыли и прочитали файлы, проверили анализ на дубли и пропуски

## Обучение и проверка модели

In [8]:
features_0 = df_0.drop(['product','id'], axis=1)
target_0 = df_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0,test_size=0.25, random_state=12345)
model_0 = LinearRegression(normalize=True)
model_0.fit(features_train_0, target_train_0)
predictions_0 = model_0.predict(features_valid_0)
rmse_0 = (mean_squared_error(predictions_0, target_valid_0))**(0.5)
average_prod_0 = predictions_0.mean()
print('RMSE первого региона', rmse_0)
print('Средний запас предсказанного сырья по первому региону:', average_prod_0)

RMSE первого региона 37.853527328872964
Средний запас предсказанного сырья по первому региону: 92.78915638280621


In [9]:
features_1 = df_1.drop(['product','id'], axis=1)
target_1 = df_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1,test_size=0.25, random_state=12345)
model_1 = LinearRegression(normalize=True)
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)
rmse_1 = (mean_squared_error(predictions_1, target_valid_1))**(0.5)
average_prod_1 = predictions_1.mean()
print('RMSE второго региона', rmse_1)
print('Средний запас предсказанного сырья по второму региону:', average_prod_1)

RMSE второго региона 0.892059264771702
Средний запас предсказанного сырья по второму региону: 69.17831957030432


In [10]:
features_2 = df_2.drop(['product','id'], axis=1)
target_2 = df_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2,test_size=0.25, random_state=12345)
model_2 = LinearRegression(normalize=True)
model_2.fit(features_train_2, target_train_2)
predictions_2 = model_2.predict(features_valid_2)
rmse_2 = (mean_squared_error(predictions_2, target_valid_2))**(0.5)
average_prod_2 = predictions_2.mean()
print('RMSE третьего региона', rmse_2)
print('Средний запас предсказанного сырья по третьему региону:', average_prod_2)

RMSE третьего региона 40.07585073246016
Средний запас предсказанного сырья по третьему региону: 94.86572480562035


### Вывод:

1) Мы разделили данные на обучающую и валидну выборки

2) Обучили модель линейной регрессии

3) Посчитали среднее количество предсказанных запасов в каждом регионе, и RMSE

## Подготовка к расчёту прибыли

Сначала сохраним данные нужные для расчетов

In [11]:
budget = 10**10
one_bar_income = 450
income_per_unit = 450*1000
mines = 200

Посчитаем объем сырья нужного для добычи без убытков

In [12]:
reg_development = budget / income_per_unit
print('Точка безубыточности:', reg_development)

Точка безубыточности: 22222.222222222223


In [13]:
break_even_point = np.ceil(reg_development / mines)

In [14]:
print('Средний объем в скважине для безубыточной добычи:', break_even_point)

Средний объем в скважине для безубыточной добычи: 112.0


Сравним нашу точку безубысточности со средним в каждом регионе 

In [15]:
def comparison(i, point):
    break_even_point = point
    if i < break_even_point:
        print('В регионе нехватает', (break_even_point-i), 'тон сырья')
    else:
        print('В регионе сырье в достатке')

In [16]:
regions=[average_prod_0, average_prod_1, average_prod_2]
row = 0
for i in regions:
    row += 1
    print('Регион №', row)
    comparison(i, break_even_point)

Регион № 1
В регионе нехватает 19.210843617193788 тон сырья
Регион № 2
В регионе нехватает 42.821680429695675 тон сырья
Регион № 3
В регионе нехватает 17.134275194379654 тон сырья


### Функция для расчета прибыли

In [17]:
def revenue (target, predictions):
    target = pd.Series(target).reset_index(drop=True)
    predictions = pd.Series(predictions).reset_index(drop=True)
    predict_sort = predictions.sort_values(ascending=False)[:mines].index
    selected = target[predict_sort]
    revenue = selected.sum() * income_per_unit - budget
    return round(revenue, 2)

In [18]:
revenue(target_valid_0, predictions_0)

3365187237.7

In [19]:
revenue(target_valid_1, predictions_1)

2415086696.68

In [20]:
revenue(target_valid_2, predictions_2)

2501283853.28

### Вывод:

Мы рассчитали кол-во барелей необходимых для безубыточной разработки - 112 штук, что болше чем во всех трех регионах

так же мы написали функцию расчета прибыли 

## Расчёт прибыли и рисков 

напишем функцию для бутстрепа

In [21]:
def bootstrap(target, probabilities, iterations, sample_size):
    state = np.random.RandomState(12345)
    values = []
    for i in range(iterations):
        target_subsample = target.sample(n=sample_size, random_state=state, replace=True)
        probs_subsample = probabilities[target_subsample.index]
        values.append(revenue(target_subsample, probs_subsample))

    values = pd.Series(values)
    mean = values.mean()
    risk = (values < 0).mean()
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    quantile_interval = (lower, upper)
    return round(mean,2), confidence_interval, round(risk*100, 2), quantile_interval

In [22]:
mean_0, interval_0, risk_0, quantile_0 = bootstrap(target_valid_0.reset_index(drop=True), predictions_0, 1000, 500)
print('Регион №1 средняя выручка = ', mean_0)
print('Регион №1 95 %ый доверительный интервал по стьюденту = ', interval_0)
print('Регион №1 95 %ый доверительный интервал по квантилям = ', quantile_0)
print(f'Регион №1 Риск убытков: {risk_0}%')

Регион №1 средняя выручка =  380613470.02
Регион №1 95 %ый доверительный интервал по стьюденту =  (364560620.985091, 396666319.050089)
Регион №1 95 %ый доверительный интервал по квантилям =  (-142942739.73875, 890976833.84825)
Регион №1 Риск убытков: 7.7%


In [23]:
mean_1, interval_1, risk_1, quantile_1 = bootstrap(target_valid_1.reset_index(drop=True), predictions_1, 1000, 500)
print('Регион №2 средняя выручка = ', mean_1)
print('Регион №2 95 %ый доверительный интервал по стьюденту = ', interval_1)
print('Регион №2 95 %ый доверительный интервал по квантилям = ', quantile_1)
print(f'Регион №2 Риск убытков: {risk_1}%')

Регион №2 средняя выручка =  478484939.59
Регион №2 95 %ый доверительный интервал по стьюденту =  (465742044.4694332, 491227834.7147268)
Регион №2 95 %ый доверительный интервал по квантилям =  (89525124.97174999, 866604745.4622499)
Регион №2 Риск убытков: 1.2%


In [24]:
mean_2, interval_2, risk_2, quantile_2 = bootstrap(target_valid_2.reset_index(drop=True), predictions_2, 1000, 500)
print('Регион №3 средняя выручка = ', mean_2)
print('Регион №3 95 %ый доверительный интервал по стьюденту = ', interval_2)
print('Регион №3 95 %ый доверительный интервал по квантилям = ', quantile_2)
print(f'Регион №3 Риск убытков: {risk_2}%')

Регион №3 средняя выручка =  330546258.02
Регион №3 95 %ый доверительный интервал по стьюденту =  (314344958.63490766, 346747557.3999924)
Регион №3 95 %ый доверительный интервал по квантилям =  (-190365733.86475, 840467403.6469997)
Регион №3 Риск убытков: 11.3%


## Вывод:

Лушчим выбором будет второй регион!

Там самый низкий риск прогореть и самая большая средняя выручка! 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован